# Creating simple RESTful using Express library

- This file explains how to create express server.
- You should use [client](12-express-client.ipynb) to connect to this server as client.
- You should use both notebooks [server](12-express-server.ipynb) and [client](12-express-client.ipynb) along with each other.

**NOTE**: Last cell in this doc will stop the server. Only run it when you are finished.

In [1]:
const path = require('path');
const cwd = process.cwd();
const envPath = path.join(cwd, 'express-server-env', '.env');
require('dotenv').config({ path: envPath });
const mongoose = require('mongoose');
const bcrypt = require('bcryptjs');
const jwt = require('jsonwebtoken');
const express = require('express');
const mime = require('mime-types');

{
  parsed: {
    JWT_SECRET: 'e9c6a7c9e7b573a3a0b9dcb7c6f58d52e9f2b4d6b8f73aaf2e80bd99f46a2b0c'
  }
}

## Important notes

The `req` object in both classic HTTP and Express.js contains essential information about the incoming request. However, Express.js provides additional features and abstractions that make it easier to work with requests and handle common scenarios.

**Classic HTTP:**

* **`req.url`:** Contains the URL path of the request.
* **`req.method`:** Contains the HTTP method of the request (e.g., GET, POST, PUT, DELETE).
* **`req.headers`:** Contains an object of HTTP headers sent with the request.

- Then we can use:
  ```javascript
  const parsedUrl = url.parse(req.url, true); // Parse the URL with query string support
  const pathname = parsedUrl.pathname;
  const query = parsedUrl.query; // as object
  ```

**Express.js:**

* **`req.query`:** Contains the query string parameters appended to the URL.
* **`req.params`:** Contains the dynamic route parameters captured by Express.js.
* **`req.body`:** Contains the parsed request body, automatically parsed by Express.js middleware like `express.json()` or `express.urlencoded()`.
* **`req.headers`:** Same as in classic HTTP, but might have additional headers added by Express.js middleware.
* **`req.cookies`:** Contains the cookies sent with the request.
* **`req.session`:** Contains session data stored for the user's session (if session middleware is used).

**Key Differences:**

* **Express.js provides additional properties** like `req.query`, `req.params`, `req.cookies`, and `req.session` to simplify common tasks and make the code more concise.
* **Express.js middleware** automatically parses the request body and sets the `req.body` property, making it easier to access the request data.
* **Classic HTTP might require more manual parsing and handling** of headers, body, and other request data.

**In summary,** the `req` object in both classic HTTP and Express.js contains essential information about the incoming request. However, Express.js provides additional features and abstractions that make it easier to work with requests and handle common scenarios.

## Initiating the express app and routes

In [2]:
const app = express();
const authRoutes = express.Router();
const userRoutes = express.Router();
let server
const ObjectId = mongoose.Types.ObjectId;
/**
 * Determines the MIME type for a given file based on its file extension.
 * 
 * @param {string} filePath - The path to the file whose MIME type is to be determined.
 * @returns {string} The MIME type of the file. If the file extension is not recognized, returns 'application/octet-stream'.
 * 
 * @description
 * This function extracts the file extension from the provided `filePath`, converts it to lowercase, and looks up the corresponding MIME type using the `mime` library.
 * If the MIME type for the given file extension is not found, it defaults to 'application/octet-stream'.
 * 
 * @example
 * const mimeType = getContentType('/path/to/file.jpg');
 * console.log(mimeType); // Outputs: 'image/jpeg'
 */
function getContentType(filePath) {
    const extname = path.extname(filePath).toLowerCase();
    return mime.lookup(extname) || 'application/octet-stream';
}

## Initiating Monggose database

In [3]:
const dbName = "nodejs-learners-package-express-db";
const dbPort = '27017'; // Set your MongoDB port
const uri = `mongodb://127.0.0.1:${dbPort}/${dbName}`; // Replace with your actual connection uri

In [4]:
const UserSchema = new mongoose.Schema({
    username: { type: String, required: true, unique: true },
    password: { type: String, required: true }
});

// Hash the password before saving
/*
Explanation:
UserSchema.pre('save', async (next) => {...}: This sets up a Mongoose middleware function that runs before 
    saving a document (pre-save hook). 
    The async keyword indicates that this function performs asynchronous operations.

if (!this.isModified('password')) return next();: Checks if the password field has been modified. 
    If the password hasn’t been changed, there’s no need to hash it again,
    so the middleware proceeds to the next function (next()). 
    This prevents unnecessary hashing if the document is being updated but the password hasn’t changed.

const salt = await bcrypt.genSalt(10);: Generates a salt using bcrypt. 
    A salt is a random value added to the password before hashing to ensure that 
    identical passwords have different hashes. 
    The 10 is the salt rounds, which determines the computational cost of the hashing. 
    More rounds mean more security but slower hashing.

this.password = await bcrypt.hash(this.password, salt);: Hashes the password with the generated salt. 
    The this.password refers to the password field of the current document. 
    The await keyword ensures that the function waits for the hashing to complete before proceeding.

next();: Moves on to the next middleware or completes the save operation if there are no more middlewares.
*/
// NOTE: do not use arrow function; in that case 'this' will not bound to the function
UserSchema.pre('save', async function (next) {
    if (!this.isModified('password')) return next();
    const salt = await bcrypt.genSalt(10);
    this.password = await bcrypt.hash(this.password, salt);
    next();
});

// Method to compare passwords
/*
Explanation:
UserSchema.methods.comparePassword = (candidatePassword) => {...}: Adds a method to the user schema. 
    This method is an instance method, which means it operates on a specific user document 
    (i.e., `this` refers to the instance of the user document).

bcrypt.compare(candidatePassword, this.password): Uses bcrypt to compare the candidatePassword 
    (the plain text password provided by a user during login) with the hashed password stored in the 
    this.password field (the hashed password from the database). 
    The bcrypt.compare function returns a promise that resolves to true if the passwords match and false otherwise.
*/
// NOTE: do not use arrow function; in that case 'this' will not bound to the function
UserSchema.methods.comparePassword = function (candidatePassword) {
    // `this.password` should be a hashed password string, and `candidatePassword` should be the plain password
    return bcrypt.compare(candidatePassword, this.password);
};

const User = mongoose.model('User', UserSchema);

[Function (anonymous)]

## Creating middlewares

In [5]:
const authMiddleware = (req, res, next) => {
    const token = req.headers['authorization']?.split(' ')[1];
    if (!token) return res.status(401).json({ message: 'Authentication is required. Provide your access token.' });

    jwt.verify(token, process.env.JWT_SECRET, (err, decoded) => {
        if (err) return res.status(403).json({ message: 'Invalid token' });
        req.user = decoded; // <-- Token payload is added to req " { id: user._id} "
        next();
    });
};
const checkUsernameMiddleware = (req, res, next) => {
    const userId = new ObjectId(req.user.id);
    User.findOne({
        _id: userId,
        username: req.params.username
    })
    .then(res => {
        next();
    })
    .catch((error) => {
        return res.status(403).json({ message: 'Forbidden: You don\'t have access to this content.' });
    });
}

## Creating primary routes

In [6]:
// Sign Up: we use json middleware to get the request body as json object here
authRoutes.post('/signup', express.json(), async (req, res) => {
    try {
        const { username, password } = req.body;

        // Check for existing username
        const existingUser = await User.findOne({ username });

        if (existingUser) {
          return res.status(400).json({ message: 'Username already exists' });
        }

        const user = new User({ username, password });
        await user.save();   

        res.status(201).json({ message: 'User created' });
    } catch (error) {
        res.status(400).json({ message: error.message });
    }
});

// Sign In: we use json middleware to get the request body as json object here
authRoutes.post('/signin', express.json(), async (req, res) => {
    try {

        const { username, password } = req.body;
        const user = await User.findOne({ username });
        if (!user || !(await user.comparePassword(password))) {
            return res.status(400).json({ message: 'Invalid credentials' });
        }
        const token = jwt.sign(
            { id: user._id.toString()},
            process.env.JWT_SECRET, 
            { expiresIn: '1h' }
        );
        res.json({ token });
    } catch (error) {
        res.status(400).json({ message: error.message });
    }
});

// Protected route example; uses authMiddleware for jwt authentication
userRoutes.get('/:username/profile', authMiddleware, checkUsernameMiddleware, (req, res) => {
    res.json({ message: 'This is your profile', userId: req.user.id.toString() });
});

userRoutes.delete('/:username/profile', authMiddleware, checkUsernameMiddleware, (req, res) => {
    const userId = new ObjectId(req.user.id);
    User.deleteOne({_id: userId})
      .then(result => {
          res.status(200).json({ message: 'Your account was deleted.'});
      })
      .catch(err => {
          res.status(404).json({ message: 'There was a problem deleting your account.'});
      });
});

[Function: router] {
  params: {},
  _params: [],
  caseSensitive: undefined,
  mergeParams: undefined,
  strict: undefined,
  stack: [
    Layer {
      handle: [Function: bound dispatch],
      name: 'bound dispatch',
      params: undefined,
      path: undefined,
      keys: [Array],
      regexp: /^\/(?:([^\/]+?))\/profile\/?$/i,
      route: [Route]
    },
    Layer {
      handle: [Function: bound dispatch],
      name: 'bound dispatch',
      params: undefined,
      path: undefined,
      keys: [Array],
      regexp: /^\/(?:([^\/]+?))\/profile\/?$/i,
      route: [Route]
    }
  ]
}

## Creating other routes

In [7]:
userRoutes.get('/:username/download', authMiddleware, checkUsernameMiddleware,
    (req, res) => {
        const fileAbsolutePath = path.join(cwd, 'express-server-downloads', 'server-download-image.png');
        const imageType = getContentType(fileAbsolutePath); // Get MIME type of image
        res.setHeader('Content-Type', imageType || 'image/jpeg'); // Set content type for image
        res.statusCode = 200; // Set status code for successful response
        
        res.sendFile(fileAbsolutePath, err => {
            if (err) {
                console.error('File send error:', err);
                res.status(err.status || 500).end();
            }
        });
    });

userRoutes.put('/:username/upload/direct', authMiddleware, checkUsernameMiddleware,
    express.raw({ type: 'image/png', limit: '10mb' }),
    (req, res) => {

        // reg.body is buffer
        if (!req.body || !req.body.length) {
            return res.status(400).json({ message: 'No file data received' });
        }
        
        const filePath = path.join(cwd, 'express-server-uploads', 'uploaded-image.png');
        // Save the file to disk
        fs.writeFile(filePath, req.body, (err) => {
            if (err) {
                return res.status(500).json({ message: 'Failed to save file', error: err });
            }
            res.status(200).json({ message: 'File uploaded successfully'});
        });
    });

userRoutes.put('/:username/upload/stream', (req, res) => {
    const filePath = path.join(cwd, 'express-server-uploads', 'uploaded-image.png');

    // Create a write stream to save the file
    const writeStream = fs.createWriteStream(filePath);

    // Pipe the request's readable stream to the write stream
    req.pipe(writeStream);

    writeStream.on('finish', () => {
        res.status(200).json({ message: 'File uploaded successfully', filePath });
    });

    writeStream.on('error', (err) => {
        res.status(500).json({ message: 'Failed to save file', error: err });
    });

    req.on('error', (err) => {
        res.status(500).json({ message: 'Failed to save file', error: err });
    });
});

[Function: router] {
  params: {},
  _params: [],
  caseSensitive: undefined,
  mergeParams: undefined,
  strict: undefined,
  stack: [
    Layer {
      handle: [Function: bound dispatch],
      name: 'bound dispatch',
      params: undefined,
      path: undefined,
      keys: [Array],
      regexp: /^\/(?:([^\/]+?))\/profile\/?$/i,
      route: [Route]
    },
    Layer {
      handle: [Function: bound dispatch],
      name: 'bound dispatch',
      params: undefined,
      path: undefined,
      keys: [Array],
      regexp: /^\/(?:([^\/]+?))\/profile\/?$/i,
      route: [Route]
    },
    Layer {
      handle: [Function: bound dispatch],
      name: 'bound dispatch',
      params: undefined,
      path: undefined,
      keys: [Array],
      regexp: /^\/(?:([^\/]+?))\/download\/?$/i,
      route: [Route]
    },
    Layer {
      handle: [Function: bound dispatch],
      name: 'bound dispatch',
      params: undefined,
      path: undefined,
      keys: [Array],
      regexp: /^\/(?:([

## Assign routers

In [8]:
// Routes
app.use('/users', userRoutes); // ./profile
app.use('/auth', authRoutes); // ./signin and ./signup

<ref *1> [Function: app] {
  _events: [Object: null prototype] { mount: [Function: onmount] },
  _eventsCount: 1,
  _maxListeners: undefined,
  setMaxListeners: [Function: setMaxListeners],
  getMaxListeners: [Function: getMaxListeners],
  emit: [Function: emit],
  addListener: [Function: addListener],
  on: [Function: addListener],
  prependListener: [Function: prependListener],
  once: [Function: once],
  prependOnceListener: [Function: prependOnceListener],
  removeListener: [Function: removeListener],
  off: [Function: removeListener],
  removeAllListeners: [Function: removeAllListeners],
  listeners: [Function: listeners],
  rawListeners: [Function: rawListeners],
  listenerCount: [Function: listenerCount],
  eventNames: [Function: eventNames],
  init: [Function: init],
  defaultConfiguration: [Function: defaultConfiguration],
  lazyrouter: [Function: lazyrouter],
  handle: [Function: handle],
  use: [Function: use],
  route: [Function: route],
  engine: [Function: engine],
  para

## Initiating the server

In [9]:
// Connect to MongoDB
mongoose.connect(uri)
    .then(() => { 
        console.log('MongoDB connected');
        //const PORT = process.env.PORT || 5678;
        const PORT = 5678;
        server = app.listen(PORT, () => console.log(`Server running on port ${PORT}`));
    })
    .catch(err => {
        console.error('An error happened during server initiation:', err);
        process.exit(1); // Optional: Exit the process if connection fails
    });


Promise { <pending> }

MongoDB connected
Server running on port 5678


## Stopping the server

In [9]:
mongoose.disconnect().then(() => {
        console.log('MongoDB connection closed');
        server?.close(() => {
            console.log('Express server closed');
            process.exit();
        });
    });


Promise { <pending> }

MongoDB connection closed
Express server closed
